In [7]:
from tabulate import tabulate
import numpy as np
from numpy.dtypes import StringDType

import random
from collections import Counter

n_parts: int = 4
n_subs: int = 2
n_periods = n_parts * n_subs

team_members: list[str] = [
    "Benjamin",
    "Vik",
    "Vigo",
    "Can",
    "Njabulo",
    "Lars",
    "Sebas",
    "Kees",
    "Hamza",
]

excl_team: list[str] = ["Hamza"]

tasks_base = [
    "Keeper",
    "Aanvaller",
    "Middenvelder",
    "Middenvelder",
    "Verdediger",
    "Verdediger",
]

headers = [f"Periode {i + 1}" for i in range(n_parts)]

keepers: list[str] = ["Can", "Vik", "Vigo", "Njabulo"]
keeper_idx = [team_members.index(keeper) for keeper in keepers]
random.shuffle(keeper_idx)

team: list[str] = [i for i in team_members if i not in excl_team]
print(team)
team_idx = [team_members.index(member) for member in team]
random.shuffle(team_idx)

n_team: int = len(team)  # Number of team members
n_tasks: int = len(tasks_base)  # Number of team positions
n_subs: int = n_team - n_tasks  # The number of subs
print(n_subs)

tasks_sub = ["Wissel" for i in range(1, n_subs + 1)]
tasks = tasks_base + tasks_sub

# Create the matrix where rows are n team members playing and columns the
# n of periods
matrix: np.array = np.full((n_team, n_periods), -1).astype(int)

['Benjamin', 'Vik', 'Vigo', 'Can', 'Njabulo', 'Lars', 'Sebas', 'Kees']
2


In [8]:
# Set the keepers
matrix[0, np.arange(len(keeper_idx)) * 2] = keeper_idx
matrix[0, np.arange(len(keeper_idx)) * 2 + 1] = keeper_idx

matrix

array([[ 1,  1,  3,  3,  2,  2,  4,  4],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1]])

In [9]:
# Set the substitutes for each period
sub_list: list = team_idx * (n_subs + 1)

for i in range(n_periods):
    for j in range(1, n_subs + 1):
        # t represents the first team member id from the list    
        t: int = sub_list[0]

        # Check if the team member is already in the column
        if np.isin(t, matrix[:,i]):
            # Take the next element in the list given t is Keeper    
            t = sub_list[1]
            # Check if team member was sub in previous period, if yes take next
            if np.isin(t, matrix[-n_subs:,i-1]):
                t = sub_list.pop(2)
            else:
                t = sub_list.pop(1)
            matrix[j * -1, i] = t
        else:
            sub_list.remove(t)
            matrix[j * -1, i] = t        

matrix

array([[ 1,  1,  3,  3,  2,  2,  4,  4],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [ 3,  4,  6,  1,  3,  5,  6,  1],
       [ 7,  2,  5,  0,  7,  4,  2,  0]])

In [10]:
# # Aanvallers for each period
# for i in range(n_periods):            
#     if (i+1) % 2 == 0:
#         # We are substituting in the period
#         # Get the previous attacker and check if he is substitute
#         p_attack = matrix[1,i-1] # get attacker from previous period

    
#         if p_attack in matrix[:,i]:
#             print(f"the current attacker {team_members[p_attack]} is now sub in period {i+1}")
#             loc_sub = np.where(matrix[:,i] == p_attack)[0][0]
#             matrix[1,i] = matrix[loc_sub, i-1]
#         else:
#             matrix[1,i] = matrix[1, i-1]
#     else:
#         unique_values, counts = np.unique(np.append(matrix[1,:], team_idx), 
#                                         return_counts=True)

#         # Sort the unique values and their counts
#         sorted_indices = np.argsort(counts)
#         sorted_values = unique_values[sorted_indices]

#         val = np.array([el for el in sorted_values if el not in matrix[:,i]])

#         matrix[1,i] = val[0]
     
# matrix

In [11]:
def optimize_position(matrix: np.array, n_periods: int, i_s: int, i_e: int):
    random.shuffle(team_idx)
    for i in range(n_periods):
        for j in range(i_s, i_e):
            if (i+1) % 2 == 0:
                # 2 midfielders 
                position = matrix[j,i-1]
            
                if position in matrix[:,i]:
                    loc_sub = np.where(matrix[:,i] == position)[0][0]
                    matrix[j,i] = matrix[loc_sub, i-1]
                else:
                    matrix[j,i] = matrix[j, i-1]
            else:
                sorted_pos = [num for (num, cnt) in 
                              Counter(np.append(matrix[i_s:i_e,:], team_idx)).most_common()]
                
                sorted_pos = np.flip(np.asarray(sorted_pos))

                val = np.array([el for el in sorted_pos if el not in matrix[:,i]])

                matrix[j,i] = val[0]
    
    return matrix


matrix = optimize_position(matrix, n_periods, 1, 2) # Attackers
matrix = optimize_position(matrix, n_periods, 2, 4) # Midfield
matrix = optimize_position(matrix, n_periods, 4, 6) # Defenders

matrix

array([[1, 1, 3, 3, 2, 2, 4, 4],
       [2, 7, 0, 5, 6, 6, 3, 3],
       [0, 0, 7, 7, 1, 1, 5, 5],
       [6, 6, 4, 4, 5, 3, 1, 6],
       [5, 5, 1, 6, 0, 0, 7, 7],
       [4, 3, 2, 2, 4, 7, 0, 2],
       [3, 4, 6, 1, 3, 5, 6, 1],
       [7, 2, 5, 0, 7, 4, 2, 0]])

In [ ]:
# Replace the index with the name of team members
# matrix[matrix == -99] = -1
matrix_str = [[team_members[matrix[i][j]] for j in range(n_periods)] for i in range(n_team)]

team_members_array = np.array(team_members)
print(matrix)
print(team_members_array)
matrix_str: np.ndarray = team_members_array[matrix]

matrix_str_condensed: np.array = np.zeros((matrix_str.shape[0],n_parts), dtype=StringDType())

# Condens the table to the number of parts
for i in range(n_parts):
    for j in range(matrix.shape[0]):
        if matrix_str[j, i *2] == matrix_str[j, i*2+1]:
            matrix_str_condensed[j, i] = matrix_str[j, i*2]
        else:
            matrix_str_condensed[j, i] = f"{matrix_str[j, i *2]} -> {matrix_str[j, i*2+1]}"

matrix_str = matrix_str_condensed.tolist()

# Add tasks row
[row.insert(0, task) for task, row in zip(tasks, matrix_str)]

print(tabulate(matrix_str, headers=headers, tablefmt="grid"))

+--------------+----------------+------------------+-----------------+------------------+
|              | Periode 1      | Periode 2        | Periode 3       | Periode 4        |
+==============+================+==================+=================+==================+
| Keeper       | Vik            | Can              | Vigo            | Njabulo          |
+--------------+----------------+------------------+-----------------+------------------+
| Aanvaller    | Vigo -> Kees   | Benjamin -> Lars | Sebas           | Can              |
+--------------+----------------+------------------+-----------------+------------------+
| Middenvelder | Benjamin       | Kees             | Vik             | Lars             |
+--------------+----------------+------------------+-----------------+------------------+
| Middenvelder | Sebas          | Njabulo          | Lars -> Can     | Vik -> Sebas     |
+--------------+----------------+------------------+-----------------+------------------+
| Verdedig